# Block 5 Exercise 2: Predicting the fare of taxi rides
Using the NY Taxi data set (see Use Case Block I) and the use case from the lecture...

In [1]:
import pandas as pd
import numpy as np
import folium

In [2]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv('../../DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

In [3]:
# a quick look at the data
X.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_day,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,...,pickup_borough_manhattan,pickup_borough_others,pickup_borough_queens,pickup_borough_staten_island,dropoff_borough_bronx,dropoff_borough_brooklyn,dropoff_borough_manhattan,dropoff_borough_others,dropoff_borough_queens,dropoff_borough_staten_island
0,-73.844311,40.721319,-73.841610,40.712278,1,15,17,1,6,2009,...,0,0,1,0,0,0,0,0,1,0
1,-74.016048,40.711303,-73.979268,40.782004,1,5,16,2,1,2010,...,1,0,0,0,0,0,1,0,0,0
2,-73.982738,40.761270,-73.991242,40.750562,2,18,0,4,8,2011,...,1,0,0,0,0,0,1,0,0,0
3,-73.987130,40.733143,-73.991567,40.758092,1,21,4,6,4,2012,...,1,0,0,0,0,0,1,0,0,0
4,-73.968095,40.768008,-73.956655,40.783762,1,9,7,2,3,2010,...,1,0,0,0,0,0,1,0,0,0


In [4]:
y.head()

0     4.5
1    16.9
2     5.7
3     7.7
4     5.3
Name: fare_amount, dtype: float64

### E 2.1 Split data into train and test data
* use data from 2009-1014 for training and 2015 for evaluation


In [7]:
print(X['pickup_year'].unique())

[2009 2010 2011 2012 2013 2014 2015]


In [20]:
lower_2015 = (X['pickup_year'] < 2015)
equal_2015 = (X['pickup_year'] == 2015)

X_train = X[lower_2015]
X_test = X[equal_2015]
y_train = y[lower_2015]
y_test = y[equal_2015]

### E 2.2 Train a Random Forest Regression Model
* train model using [RF regression](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* optimize the model parameters (#trees, depth, vars per split, ...) and evaluate the [MSE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error) on train and test data
* Analyze the model error: are there typical settings where the model fails?

In [95]:
# baseline with 100 trees, no max depth, minimum number of samples required to split = 2
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('Baseline of RandomForestRegressor score is {}'.format(score))
print('MSE: {}, {} trees, no max. depth, {} min_samples_split'.format(mean_squared_error(y_test, y_pred), 100, 2))

Baseline of RandomForestRegressor score is 0.8371229656804402
MSE: 21.338071707051636, 100 trees, no max. depth, 2 min_samples_split


In [98]:
trees, depth, split = 50, 3, 2
rfr = RandomForestRegressor(n_estimators=trees, max_depth=depth, min_samples_split=split)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('RF-Regressor score is {}'.format(score))
print('MSE: {}, {} trees, max. depth: {}, {} min_samples_split'.format(
    mean_squared_error(y_test, y_pred), trees, depth, split))

RF-Regressor score is 0.730359350028625
MSE: 35.32487896935059, 50 trees, max. depth: 3, 2 min_samples_split


In [99]:
trees, depth, split = 50, 5, 2
rfr = RandomForestRegressor(n_estimators=trees, max_depth=depth, min_samples_split=split)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('RF-Regressor score is {}'.format(score))
print('MSE: {}, {} trees, max. depth: {}, {} min_samples_split'.format(
    mean_squared_error(y_test, y_pred), trees, depth, split))

RF-Regressor score is 0.7987640068731401
MSE: 26.3633732608123, 50 trees, max. depth: 5, 2 min_samples_split


In [100]:
trees, depth, split = 50, 5, 5
rfr = RandomForestRegressor(n_estimators=trees, max_depth=depth, min_samples_split=split)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('RF-Regressor score is {}'.format(score))
print('MSE: {}, {} trees, max. depth: {}, {} min_samples_split'.format(
    mean_squared_error(y_test, y_pred), trees, depth, split))

RF-Regressor score is 0.7966069944011107
MSE: 26.645957524416115, 50 trees, max. depth: 5, 5 min_samples_split


In [101]:
trees, depth, split = 75, 5, 5
rfr = RandomForestRegressor(n_estimators=trees, max_depth=depth, min_samples_split=split)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('RF-Regressor score is {}'.format(score))
print('MSE: {}, {} trees, max. depth: {}, {} min_samples_split'.format(
    mean_squared_error(y_test, y_pred), trees, depth, split))

RF-Regressor score is 0.796875883199224
MSE: 26.610731143488106, 75 trees, max. depth: 5, 5 min_samples_split


In [102]:
trees, depth, split = 10, 5, 5
rfr = RandomForestRegressor(n_estimators=trees, max_depth=depth, min_samples_split=split)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('RF-Regressor score is {}'.format(score))
print('MSE: {}, {} trees, max. depth: {}, {} min_samples_split'.format(
    mean_squared_error(y_test, y_pred), trees, depth, split))

RF-Regressor score is 0.7952178257042057
MSE: 26.827948689662854, 10 trees, max. depth: 5, 5 min_samples_split


In [103]:
trees, depth, split = 10, 10, 5
rfr = RandomForestRegressor(n_estimators=trees, max_depth=depth, min_samples_split=split)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('RF-Regressor score is {}'.format(score))
print('MSE: {}, {} trees, max. depth: {}, {} min_samples_split'.format(
    mean_squared_error(y_test, y_pred), trees, depth, split))

RF-Regressor score is 0.8255425179891884
MSE: 22.855194266828107, 10 trees, max. depth: 10, 5 min_samples_split


In [105]:
trees, depth, split = 10, 10, 10
rfr = RandomForestRegressor(n_estimators=trees, max_depth=depth, min_samples_split=split)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('RF-Regressor score is {}'.format(score))
print('MSE: {}, {} trees, max. depth: {}, {} min_samples_split'.format(
    mean_squared_error(y_test, y_pred), trees, depth, split))

RF-Regressor score is 0.8245152105296945
MSE: 22.989778987918168, 10 trees, max. depth: 10, 10 min_samples_split


In [151]:
# The higher, the more important the feature. 
idx = list(map(lambda x: ('%.3f' % x),rfr.feature_importances_)).index(max)
print('Decision was made by "{}".'.format(X.columns[idx]))

Decision was made by "trip_distance".


In [80]:
path = rfr.decision_path(X_train)

In [83]:
path

(<372110x50162 sparse matrix of type '<class 'numpy.int64'>'
 	with 204581944 stored elements in Compressed Sparse Row format>,
 array([    0,  1025,  2022,  3017,  3984,  4981,  5980,  7021,  8016,
         9007, 10010, 10989, 11942, 12919, 13916, 14923, 15942, 16939,
        17920, 18927, 19940, 20961, 21966, 22999, 24000, 24975, 26014,
        27043, 28054, 29069, 30030, 31003, 32044, 33009, 33984, 34983,
        36032, 37053, 38056, 39071, 40070, 41057, 42088, 43077, 44034,
        45037, 46074, 47085, 48104, 49115, 50162]))